In [1]:
import os
import sys
sys.path.append('..')
LIB_PATH = os.getcwd() + "/art_library"
sys.path.append(LIB_PATH)
print(*sys.path, sep='\n')

/home/lukec/workspace/baard_exp2/experiments
/home/lukec/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles
/home/lukec/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/workspace/baard_exp2/venv/lib/python3.6/site-packages
/home/lukec/workspace/baard_exp2/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython
..
/home/lukec/workspace/baard_exp2/experiments/art_library


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd

In [4]:
from models.mnist import BaseModel
from models.cifar10 import  Resnet
from models.torch_util import predict_numpy
from defences.util import acc_on_adv
from defences.baard import ApplicabilityStage, ReliabilityStage, DecidabilityStage, BAARDOperator

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [6]:
dnn = BaseModel(use_prob=True).to(device)

In [7]:
f_dnn = os.path.join('..', 'results', 'mnist_200.pt')
dnn.load_state_dict(torch.load(f_dnn, map_location=device))

<All keys matched successfully>

In [8]:
# Create BAARD
f_json_param = os.path.join('..', 'params', 'baard_mnist_3.json')
with open(f_json_param) as j:
    baard_param = json.load(j)
print('Param:', baard_param)

n_classes = 10

Param: {'k_re': 10, 'k_de': 100, 'q1': 1.0, 'q2': 0.99, 'q3': 0.99, 'sequence': [True, True, True]}


In [9]:
stages_resnet = []
stages_resnet.append(ApplicabilityStage(n_classes=n_classes, quantile=baard_param['q1'], verbose=0))
stages_resnet.append(ReliabilityStage(n_classes=n_classes, k=baard_param['k_re'], quantile=baard_param['q2'], verbose=0))
stages_resnet.append(DecidabilityStage(n_classes=n_classes, k=baard_param['k_de'], quantile=baard_param['q3'], verbose=0))
detector_resnet = BAARDOperator(stages=stages_resnet)

In [10]:
# Load pretrained parameters
f_pre_data = os.path.join('..', 'result_0', 'mnist_dnn_baard_s1_train_data.pt')
obj = torch.load(f_pre_data)
X_train_s1 = obj['X_s1']
X_train = obj['X']
y_train = obj['y']
print(X_train_s1.shape, X_train.shape, y_train.shape)

(59824, 1, 28, 28) (59824, 1, 28, 28) (59824,)


In [11]:
f_thresh_resnet = os.path.join('..', 'result_0', 'mnist_dnn_baard_threshold.pt')
detector_resnet.fit(X=X_train, y=y_train, X_s1=X_train_s1)
detector_resnet.load(f_thresh_resnet)

Load from: ../result_0/mnist_dnn_baard_threshold.pt


In [13]:
def get_data(data, model_name, att, e, d='adv'):
    f = '{}_{}_{}_{}_{}.npy'.format(data, model_name, att, e, d)
    p = os.path.join('..', 'results', f)
    return np.load(p)

In [15]:
att = 'apgd'
e = 0.063
data = 'mnist'
model_name = 'basic'
X = get_data(data, model_name, att, e, d='x')[:500]
adv = get_data(data, model_name, att, e)[:500]
y = get_data(data, model_name, att, e, d='y')[:500]
pred_adv = predict_numpy(dnn, adv, device)[:500]

In [16]:
np.mean(pred_adv == y)

0.794

In [17]:
s1 = detector_resnet.stages[0]
s2 = detector_resnet.stages[1]
s3 = detector_resnet.stages[2]

In [18]:
ks_s2 = np.arange(31, step=2)[1:]
acc_s2 = np.zeros_like(ks_s2, dtype=np.float)
fprs_s2 = np.zeros_like(ks_s2, dtype=np.float)

for i, k in enumerate(ks_s2):
    s2.k = k
    labelled_adv = s2.predict(adv, pred_adv)
    acc = acc_on_adv(pred_adv, y, labelled_adv)
    acc_s2[i] = acc

    labelled_x = s2.predict(X, y)
    fpr = np.mean(labelled_x)
    fprs_s2[i] = fpr

    print('[{:2d}] {:.4f} {:.4f}'.format(k, acc, fpr))

[ 2] 0.8100 0.0060
[ 4] 0.8140 0.0140
[ 6] 0.8160 0.0180
[ 8] 0.8180 0.0200
[10] 0.8180 0.0200
[12] 0.8180 0.0220
[14] 0.8180 0.0220
[16] 0.8200 0.0260
[18] 0.8240 0.0300
[20] 0.8320 0.0340
[22] 0.8320 0.0360
[24] 0.8320 0.0380
[26] 0.8340 0.0380
[28] 0.8340 0.0400
[30] 0.8340 0.0480


In [22]:
data_s2_ = {
    'k': ks_s2,
    'acc': acc_s2,
    'fpr': fprs_s2
}
df_s2 = pd.DataFrame(data_s2_)
print(df_s2)
f = '{}_{}_s2_r_{}_{}.csv'.format(data, 'dnn', att, int(e * 1000))
print(f)
df_s2.to_csv(f, index=False)

     k    acc    fpr
0    2  0.810  0.006
1    4  0.814  0.014
2    6  0.816  0.018
3    8  0.818  0.020
4   10  0.818  0.020
5   12  0.818  0.022
6   14  0.818  0.022
7   16  0.820  0.026
8   18  0.824  0.030
9   20  0.832  0.034
10  22  0.832  0.036
11  24  0.832  0.038
12  26  0.834  0.038
13  28  0.834  0.040
14  30  0.834  0.048
mnist_dnn_s2_r_apgd_63.csv


In [23]:
ks_s3 = np.arange(10, 200, step=10)
acc_s3 = np.zeros_like(ks_s3, dtype=np.float)
fprs_s3 = np.zeros_like(ks_s3, dtype=np.float)

for i, k in enumerate(ks_s3):
    s3.k = k
    labelled_adv = s2.predict(adv, pred_adv)
    acc = acc_on_adv(pred_adv, y, labelled_adv)
    acc_s3[i] = acc

    labelled_x = s3.predict(X, y)
    fpr = np.mean(labelled_x)
    fprs_s3[i] = fpr
    
    print('[{:3d}] {:.4f} {:.4f}'.format(k, acc, fpr))

[ 10] 0.8340 0.0200
[ 20] 0.8340 0.0200
[ 30] 0.8340 0.0180
[ 40] 0.8340 0.0200
[ 50] 0.8340 0.0180
[ 60] 0.8340 0.0260
[ 70] 0.8340 0.0260
[ 80] 0.8340 0.0300
[ 90] 0.8340 0.0300
[100] 0.8340 0.0300
[110] 0.8340 0.0360
[120] 0.8340 0.0400
[130] 0.8340 0.0380
[140] 0.8340 0.0420
[150] 0.8340 0.0420
[160] 0.8340 0.0440
[170] 0.8340 0.0460
[180] 0.8340 0.0460
[190] 0.8340 0.0460


In [24]:
data_s3 = {
    'k': ks_s3,
    'acc': acc_s3,
    'fpr': fprs_s3
}
df_s3 = pd.DataFrame(data_s3)

In [25]:
f = '{}_{}_s3_r_{}_{}.csv'.format(data, 'dnn', att, int(e * 1000))
print(f)
df_s3.to_csv(f, index=False)

mnist_dnn_s3_r_apgd_63.csv
